In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!rsync -r --progress ./drive/MyDrive/MoNuSegTestData.zip .
!rsync -r --progress ./drive/MyDrive/MoNuSegTrainingData.zip .

sending incremental file list
MoNuSegTestData.zip
     41,620,178 100%   72.77MB/s    0:00:00 (xfr#1, to-chk=0/1)
sending incremental file list
MoNuSegTrainingData.zip
     84,126,760 100%   60.62MB/s    0:00:01 (xfr#1, to-chk=0/1)


In [3]:
!unzip MoNuSegTestData.zip
!unzip MoNuSegTrainingData.zip
!rm MoNuSegTestData.zip
!rm MoNuSegTrainingData.zip

Archive:  MoNuSegTestData.zip
   creating: MoNuSegTestData/
  inflating: MoNuSegTestData/TCGA-2Z-A9J9-01A-01-TS1.tif  
  inflating: MoNuSegTestData/TCGA-2Z-A9J9-01A-01-TS1.xml  
  inflating: MoNuSegTestData/TCGA-44-2665-01B-06-BS6.tif  
  inflating: MoNuSegTestData/TCGA-44-2665-01B-06-BS6.xml  
  inflating: MoNuSegTestData/TCGA-69-7764-01A-01-TS1.tif  
  inflating: MoNuSegTestData/TCGA-69-7764-01A-01-TS1.xml  
  inflating: MoNuSegTestData/TCGA-A6-6782-01A-01-BS1.tif  
  inflating: MoNuSegTestData/TCGA-A6-6782-01A-01-BS1.xml  
  inflating: MoNuSegTestData/TCGA-AC-A2FO-01A-01-TS1.tif  
  inflating: MoNuSegTestData/TCGA-AC-A2FO-01A-01-TS1.xml  
  inflating: MoNuSegTestData/TCGA-AO-A0J2-01A-01-BSA.tif  
  inflating: MoNuSegTestData/TCGA-AO-A0J2-01A-01-BSA.xml  
  inflating: MoNuSegTestData/TCGA-CU-A0YN-01A-02-BSB.tif  
  inflating: MoNuSegTestData/TCGA-CU-A0YN-01A-02-BSB.xml  
  inflating: MoNuSegTestData/TCGA-EJ-A46H-01A-03-TSC.tif  
  inflating: MoNuSegTestData/TCGA-EJ-A46H-01A-03-TSC.xm

In [4]:
!pip install xmltodict

In [5]:
import glob, os, cv2
import numpy as np
import xmltodict
import shutil

def mask2impath_train(path):
  return path.replace("Annotations", "Tissue Images").replace(".xml", ".tif")
def mask2impath_test(path):
  return path.replace(".xml", ".tif")

def create_mask(data_path, imwrite_path):

  #data_path = "./MoNuSegTrainingData/"
  annos = glob.glob(os.path.join(data_path, "*.xml"))
  #imwrite_path = "MoNuSegTrainingData/annot_jpg"
  try:
    shutil.rmtree(imwrite_path)
  except:
    pass

  os.mkdir(imwrite_path)

  for anno_path in annos:
    print(anno_path)
    cnts = []
    with open(anno_path, "r") as f:
      anno = xmltodict.parse(f.read())

      try:
        regions = anno["Annotations"]["Annotation"]["Regions"]["Region"]
      except:
        regions = anno["Annotations"]["Annotation"][0]["Regions"]["Region"]

      for region in regions:
        cnt = []
        for point in region["Vertices"]["Vertex"]:
          cnt.append([float(point["@X"]), float(point["@Y"])])
        cnt = np.array(cnt).reshape((-1,1,2)).astype(np.int32)
        cnts.append(cnt)

    if "Annotations" in data_path:
      img = cv2.imread(mask2impath_train(anno_path))
    else:
      img = cv2.imread(mask2impath_test(anno_path))

    img_zero = np.zeros_like(img)
    img_mask = cv2.drawContours(img_zero, cnts, -1, (255,255,255), 3)

    img = cv2.resize(img, (256,256))
    img_mask = cv2.resize(img_mask, (256, 256))

    if "Annotations" in data_path:
      imwrite_path2 = os.path.join(imwrite_path, mask2impath_train(anno_path).split("/")[-1].replace(".tif", ".png"))
    else:
      imwrite_path2 = os.path.join(imwrite_path, mask2impath_test(anno_path).split("/")[-1].replace(".tif", ".png"))
    #imwrite_path2 = os.path.join(imwrite_path, mask2impath(anno_path).split("/")[-1].replace(".tif", ".png"))
    print("path: ", imwrite_path2)
    cv2.imwrite(imwrite_path2, img_mask)


create_mask(data_path = "./MoNuSegTrainingData/Annotations", imwrite_path = "MoNuSegTrainingData/trainB")
create_mask(data_path = "./MoNuSegTestData/", imwrite_path = "MoNuSegTestData/valB")

./MoNuSegTrainingData/Annotations/TCGA-21-5786-01Z-00-DX1.xml
path:  MoNuSegTrainingData/trainB/TCGA-21-5786-01Z-00-DX1.png
./MoNuSegTrainingData/Annotations/TCGA-AR-A1AS-01Z-00-DX1.xml
path:  MoNuSegTrainingData/trainB/TCGA-AR-A1AS-01Z-00-DX1.png
./MoNuSegTrainingData/Annotations/TCGA-DK-A2I6-01A-01-TS1.xml
path:  MoNuSegTrainingData/trainB/TCGA-DK-A2I6-01A-01-TS1.png
./MoNuSegTrainingData/Annotations/TCGA-G9-6362-01Z-00-DX1.xml
path:  MoNuSegTrainingData/trainB/TCGA-G9-6362-01Z-00-DX1.png
./MoNuSegTrainingData/Annotations/TCGA-B0-5711-01Z-00-DX1.xml
path:  MoNuSegTrainingData/trainB/TCGA-B0-5711-01Z-00-DX1.png
./MoNuSegTrainingData/Annotations/TCGA-18-5592-01Z-00-DX1.xml
path:  MoNuSegTrainingData/trainB/TCGA-18-5592-01Z-00-DX1.png
./MoNuSegTrainingData/Annotations/TCGA-HE-7129-01Z-00-DX1.xml
path:  MoNuSegTrainingData/trainB/TCGA-HE-7129-01Z-00-DX1.png
./MoNuSegTrainingData/Annotations/TCGA-38-6178-01Z-00-DX1.xml
path:  MoNuSegTrainingData/trainB/TCGA-38-6178-01Z-00-DX1.png
./MoNuSe

In [6]:
import glob
from PIL import Image
import os
import cv2

def tif2png(data_path, imwrite_path):
  #data_path = "./MoNuSegTrainingData/Tissue Images/"
  tiff_path = glob.glob(os.path.join(data_path, "*.tif"))
  #imwrite_path = "./MoNuSegTrainingData/images_jpg/"

  try:
    shutil.rmtree(imwrite_path)
  except:
    pass

  os.mkdir(imwrite_path)

  for tif in tiff_path:
    img = cv2.imread(tif)
    img = cv2.resize(img, (256, 256))
    imwrite_path2 = os.path.join(imwrite_path, tif.split("/")[-1].replace(".tif", ".png"))
    cv2.imwrite(imwrite_path2, img)

tif2png(data_path = "./MoNuSegTrainingData/Tissue Images/", imwrite_path = "./MoNuSegTrainingData/trainA/")
tif2png(data_path = "./MoNuSegTestData/", imwrite_path = "./MoNuSegTestData/valA/")

In [7]:
from PIL import Image
import glob


def merge_images(rgb_path, mask_path, merged_path):

  mask_name_list = {}
  rgb_name_list = {}

  #rgb_path = "./MoNuSegTrainingData/images_jpg/*.jpg"
  #mask_path = "./MoNuSegTrainingData/annot_jpg/*.png"
  #merged_path = "./MoNuSegTrainingData/train/"
  try:
    os.mkdir(merged_path)
  except:
    pass

  for i in glob.glob(rgb_path):
    key_name = (i.split("/")[-1]).split(".")[-2]
    rgb_name_list[key_name] = Image.open(i)


  for j in glob.glob(mask_path):
    #print("Key: ", key_name)
    key_name = (j.split("/")[-1]).split(".")[-2]
    mask_name_list[key_name] = Image.open(j)

  for k in mask_name_list:
    if k in rgb_name_list:
      new_image = Image.new("RGB", (512, 256), (250, 250, 250))
      new_image.paste(rgb_name_list[k], (0,0))
      new_image.paste(mask_name_list[k], (256, 0))
      new_image.save(merged_path + str(k) + ".png", "PNG")

merge_images(rgb_path = "./MoNuSegTrainingData/trainA/*.png", mask_path = "./MoNuSegTrainingData/trainB/*.png", merged_path = "./MoNuSegTrainingData/train/")
merge_images(rgb_path = "./MoNuSegTestData/valA/*.png", mask_path = "./MoNuSegTestData/valB/*.png", merged_path = "./MoNuSegTestData/test/")

In [8]:
try:
  shutil.rmtree("pix2pixData")
except:
  pass

os.mkdir("pix2pixData")
os.mkdir("pix2pixData/train")
os.mkdir("pix2pixData/val")


try:
  shutil.rmtree("cycleGanData")
except:
  pass

os.mkdir("cycleGanData")
os.mkdir("cycleGanData/train")
os.mkdir("cycleGanData/val")

In [9]:
!rsync -r --progress ./MoNuSegTrainingData/train ./pix2pixData/train
!rsync -r --progress ./MoNuSegTestData/test ./pix2pixData/val
###########################################################################
!rsync -r --progress ./MoNuSegTrainingData/trainA/ ./cycleGanData/train/trainA/
!rsync -r --progress ./MoNuSegTestData/valA/ ./cycleGanData/val/valA/
!rsync -r --progress ./MoNuSegTrainingData/trainB/ ./cycleGanData/train/trainB/
!rsync -r --progress ./MoNuSegTestData/valB/ ./cycleGanData/val/valB/

sending incremental file list
train/
train/TCGA-18-5592-01Z-00-DX1.png
        200,268 100%  159.74MB/s    0:00:00 (xfr#1, to-chk=29/31)
train/TCGA-21-5784-01Z-00-DX1.png
        198,458 100%  189.26MB/s    0:00:00 (xfr#2, to-chk=28/31)
train/TCGA-21-5786-01Z-00-DX1.png
        213,898 100%  101.99MB/s    0:00:00 (xfr#3, to-chk=27/31)
train/TCGA-38-6178-01Z-00-DX1.png
        205,330 100%   65.27MB/s    0:00:00 (xfr#4, to-chk=26/31)
train/TCGA-49-4488-01Z-00-DX1.png
        209,155 100%   49.87MB/s    0:00:00 (xfr#5, to-chk=25/31)
train/TCGA-50-5931-01Z-00-DX1.png
        224,308 100%   53.48MB/s    0:00:00 (xfr#6, to-chk=24/31)
train/TCGA-A7-A13E-01Z-00-DX1.png
        197,585 100%   37.69MB/s    0:00:00 (xfr#7, to-chk=23/31)
train/TCGA-A7-A13F-01Z-00-DX1.png
        198,355 100%   31.53MB/s    0:00:00 (xfr#8, to-chk=22/31)
train/TCGA-AR-A1AK-01Z-00-DX1.png
        204,295 100%   27.83MB/s    0:00:00 (xfr#9, to-chk=21/31)
train/TCGA-AR-A1AS-01Z-00-DX1.png
        221,113 100%   26.36M

In [10]:
!git clone https://github.com/mahmoodlab/NucleiSegmentation.git

Cloning into 'NucleiSegmentation'...
remote: Enumerating objects: 148, done.
remote: Total 148 (delta 0), reused 0 (delta 0), pack-reused 148
Receiving objects: 100% (148/148), 843.70 KiB | 2.14 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [11]:
%cd NucleiSegmentation

/content/NucleiSegmentation


In [16]:
try:
  shutil.rmtree("./checkpoints")
except:
  pass

In [13]:
!pip install -r ./requirements.txt
!pip install scipy==1.1.0

     |████████████████████████████████| 686kB 13.4MB/s 
     |████████████████████████████████| 204kB 33.1MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp36-none-any.whl size=655252 sha256=04ac3abaf72c6c44590bc64b978fbf97f926da648fa30e326b1407f2e5c13ab6
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5713 sha256=0540432219f6a7d3f54e874d01469fd37823047a226ca670c59f02bc2c1badad
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom torchfile
     |████████████████████████████████| 31.2MB 111kB/s 
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uni

In [14]:
!find . -name "*.pyc" -exec rm -f {} \;

In [22]:
!python train.py --dataroot "../pix2pixData/" --name CASE1  \
  --gpu_ids 0 --display_id 0 \
  --niter 100 --niter_decay 100 \
  --pool_size 64 --loadSize 256 --fineSize 256\
  --save_epoch_freq 100 \

----------------- Options ---------------
                batchSize: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                 dataroot: ../pix2pixData/               	[default: None]
             dataset_mode: aligned                       
             display_freq: 400                           
               display_id: 0                             	[default: 1]
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
              epoch_count: 1                             
                 fineSize: 256                           
                  gpu_ids: 0                             
                init_gain: 0.02                          
 

In [ ]:
!python train.py --dataroot "../cycleGanData/train" --name CASE2  \
  --gpu_ids 0 --display_id 0 \
  --niter 100 --niter_decay 100 \
  --pool_size 64 --loadSize 256 --fineSize 256\
  --model "cycle_gan" --dataset_mode "unaligned" --which_direction "AtoB"\
  --save_epoch_freq 100

----------------- Options ---------------
                batchSize: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                 dataroot: ../cycleGanData/train         	[default: None]
             dataset_mode: unaligned                     	[default: aligned]
             display_freq: 400                           
               display_id: 0                             	[default: 1]
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
              epoch_count: 1                             
                 fineSize: 256                           
                  gpu_ids: 0                             
                init_gain: 0.02         

In [ ]:
!python train.py --dataroot "../pix2pixData/" --name CASE3  \
  --gpu_ids 0 --display_id 0 \
  --niter 100 --niter_decay 100 \
  --pool_size 64 --loadSize 256 --fineSize 256\
  --init_type "xavier"\
  --save_epoch_freq 100

----------------- Options ---------------
                batchSize: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                 dataroot: ../pix2pixData/               	[default: None]
             dataset_mode: aligned                       
             display_freq: 400                           
               display_id: 0                             	[default: 1]
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
              epoch_count: 1                             
                 fineSize: 256                           
                  gpu_ids: 0                             
                init_gain: 0.02                          
 

In [ ]:
!python train.py --dataroot "../cycleGanData/train" --name CASE4  \
  --gpu_ids 0 --display_id 0 \
  --niter 100 --niter_decay 100 \
  --pool_size 64 --loadSize 256 --fineSize 256\
  --model "cycle_gan" --dataset_mode "unaligned" --which_direction "AtoB"\
  --init_type "xavier"\
  --save_epoch_freq 100

----------------- Options ---------------
                batchSize: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                 dataroot: ../cycleGanData/train         	[default: None]
             dataset_mode: unaligned                     	[default: aligned]
             display_freq: 400                           
               display_id: 0                             	[default: 1]
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
              epoch_count: 1                             
                 fineSize: 256                           
                  gpu_ids: 0                             
                init_gain: 0.02         

In [ ]:
!ls checkpoints

CASE1  CASE2  CASE3  CASE4


In [ ]:
%cd ..

/content


In [ ]:
!rsync -r --progress ./NucleiSegmentation ./drive/MyDrive/

sending incremental file list
NucleiSegmentation/
NucleiSegmentation/README.md
          3,624 100%    0.00kB/s    0:00:00 (xfr#1, to-chk=505/507)
NucleiSegmentation/environment.yml
            249 100%   30.40kB/s    0:00:00 (xfr#2, to-chk=504/507)
NucleiSegmentation/requirements.txt
             64 100%    7.81kB/s    0:00:00 (xfr#3, to-chk=503/507)
NucleiSegmentation/test.py
          1,310 100%  142.14kB/s    0:00:00 (xfr#4, to-chk=502/507)
NucleiSegmentation/train.py
          2,359 100%  255.97kB/s    0:00:00 (xfr#5, to-chk=501/507)
NucleiSegmentation/.git/
NucleiSegmentation/.git/HEAD
             23 100%    0.06kB/s    0:00:00 (xfr#6, to-chk=491/507)
NucleiSegmentation/.git/config
            277 100%    0.70kB/s    0:00:00 (xfr#7, to-chk=490/507)
NucleiSegmentation/.git/description
             73 100%    0.18kB/s    0:00:00 (xfr#8, to-chk=489/507)
NucleiSegmentation/.git/index
          5,876 100%   14.83kB/s    0:00:00 (xfr#9, to-chk=488/507)
NucleiSegmentation/.git/packed-r

In [ ]:
!du -sh NucleiSegmentation/

4.1G	NucleiSegmentation/
